The **Part 1** of the project is divided into three sections:

1- Feature Extraction (Using SIFT)

2- Outlier Removal (Using RANSAC)

3- Computing the Homographies (Using DLT)


**pip install opencv-python**

**pip install opencv-contrib-python**

**Section 1**

In [1]:
#Imports
import numpy as np
import cv2
import os

In [2]:
#Display the video
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
while(capture.isOpened()):
    ret, frame = capture.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  ##press q if you want the video to stop 
        break
capture.release()
cv2.destroyAllWindows()

In [3]:
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
frameNr = 0
sift = cv2.SIFT_create()
while True:
    success, frame = capture.read()
    if success:
        gray= cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray,None)
        
        break



In [6]:
# how to obtain the coordinate values from the keypoints
x = kp[2].pt[0]
y = kp[2].pt[1]
print(x,y)

3.9425272941589355 50.597808837890625


128